## Download a drilling campaign object and save it in CSV format

This example shows how to download a drilling-campaign object from an Evo workspace and how to construct CSV files from the data.

> **Note:** This notebook requires an existing drilling campaign object in your Evo workspace. 
>
> If you don't have one yet, run the `create-a-drilling-campaign/sdk-examples.ipynb` notebook first to create a drilling campaign object that you can then download with this notebook.

### Requirements

You must have a Seequent account with the Evo entitlement to use this notebook.

The following parameters must be provided:

- The client ID of your Evo application.
- The callback/redirect URL of your Evo application.

To obtain these app credentials, refer to the [Apps and tokens guide](https://developer.seequent.com/docs/guides/getting-started/apps-and-tokens) in the Seequent Developer Portal.

In [ ]:
import pandas as pd

from evo.colormaps import ColormapAPIClient
from evo.notebooks import FeedbackWidget, ServiceManagerWidget
from evo.objects import ObjectAPIClient

# Evo app credentials
client_id = "<your-client-id>"  # Replace with your client ID
redirect_url = "<your-redirect-url>"  # Replace with your redirect URL

manager = await ServiceManagerWidget.with_auth_code(
    redirect_url=redirect_url,
    client_id=client_id,
).login()

### Use the Evo Python SDK to create an object client and a data client

In [ ]:
# The object client will manage your auth token and Geoscience Object API requests.
object_client = ObjectAPIClient(manager.get_environment(), manager.get_connector())

# The colormap client will manage your colormap API requests.
colormap_client = ColormapAPIClient(manager.get_environment(), manager.get_connector())

# The data client will manage saving your data as Parquet and publishing your data to Evo storage.
data_client = object_client.get_data_client(manager.cache)

### Display all drilling campaigns in the workspace

In [ ]:
import ipywidgets as widgets
from IPython.display import display

all_objects = await object_client.list_all_objects()

# Filter drilling campaigns
drilling_campaigns = [obj for obj in all_objects if "drilling-campaign" in obj.schema_id.sub_classification]

if len(drilling_campaigns) == 0:
    print("No drilling campaigns found.")
else:
    # Create dropdown options with name as label and ID as value
    dropdown_options = [(obj.name.removesuffix(".json"), str(obj.id)) for obj in drilling_campaigns]

    drilling_campaign_dropdown = widgets.Dropdown(
        options=dropdown_options,
        description="Drilling campaign:",
        disabled=False,
        style={"description_width": "initial"},
        layout=widgets.Layout(width="600px"),
    )

    print(f"Found {len(drilling_campaigns)} drilling campaign(s). Please select one:")
    display(drilling_campaign_dropdown)

Select the drilling campaign object from the dropdown above. 

The cell below downloads the object metadata and prints the number of holes in the drilling campaign.

In [ ]:
import json
from datetime import datetime
from uuid import UUID

from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer


def serialize_for_display(obj):
    """Convert an object's attributes to a JSON-serializable dictionary."""
    if isinstance(obj, UUID):
        return str(obj)
    elif isinstance(obj, datetime):
        return obj.isoformat()
    elif isinstance(obj, list):
        return [serialize_for_display(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: serialize_for_display(value) for key, value in obj.items()}
    elif hasattr(obj, "__dict__"):
        return {key: serialize_for_display(value) for key, value in vars(obj).items()}
    else:
        return obj


object_id = drilling_campaign_dropdown.value
output_filename = "drilling_campaign.xlsx"

downloaded_object = await object_client.download_object_by_id(object_id=object_id, version=None)

metadata = downloaded_object.metadata
downloaded_dict = downloaded_object.as_dict()

print("\nDrilling Campaign Structure:\n")
serialized = serialize_for_display(downloaded_dict)
print(highlight(json.dumps(serialized, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

number_of_holes = downloaded_dict["hole_id"]["table"]["length"]
print(f"\nThe drilling campaign contains {number_of_holes} holes.")

Download the drilling campaign and save as an Excel file.

In [ ]:
def download_table(table_info, fb=None):
    if fb is None:
        fb = FeedbackWidget("Downloading unknown table")
    return data_client.download_table(object_id=metadata.id, version_id=None, table_info=table_info, fb=fb)


df = pd.DataFrame()

# Use the data client to download the coordinate data.
hole_indices = (
    await download_table(
        downloaded_dict["hole_id"]["values"],
        fb=FeedbackWidget(f"Downloading hole indices data as '{downloaded_dict['hole_id']['values']['data']}'"),
    )
).to_pandas()
index_to_name_map = (
    await download_table(
        downloaded_dict["hole_id"]["table"],
        fb=FeedbackWidget(
            f"Downloading hole index to name map data as '{downloaded_dict['hole_id']['table']['data']}'"
        ),
    )
).to_pandas()
names = pd.DataFrame({"key": hole_indices["data"]}).merge(index_to_name_map, on="key", how="left")["value"]

collar_locations = (
    await download_table(
        downloaded_dict["planned"]["collar"]["coordinates"],
        fb=FeedbackWidget(
            f"Downloading collar locations data as '{downloaded_dict['planned']['collar']['coordinates']['data']}'"
        ),
    )
).to_pandas()
hole_lengths = (
    await download_table(
        downloaded_dict["planned"]["collar"]["distances"],
        fb=FeedbackWidget(
            f"Downloading collar hole distances table as '{downloaded_dict['planned']['collar']['distances']['data']}'"
        ),
    )
).to_pandas()
chunk_data = (
    await download_table(
        downloaded_dict["planned"]["collar"]["holes"],
        fb=FeedbackWidget(
            f"Downloading collar chunks data as '{downloaded_dict['planned']['collar']['holes']['data']}'"
        ),
    )
).to_pandas()

attributes = []
# Use the data client to download the attribute data and merge it with the coordinates data.
for attribute in downloaded_dict["planned"]["collar"]["attributes"]:
    attribute_name = attribute["name"]
    attribute_type = attribute["attribute_type"]

    # Download the attribute data. Every attribute has a 'values' data file.
    values_data = (
        await download_table(
            table_info=attribute["values"],
            fb=FeedbackWidget(
                f"Downloading attribute '{attribute_name}' values data as '{attribute['values']['data']}'"
            ),
        )
    ).to_pandas()
    attributes.append(values_data)

df = pd.concat([names, collar_locations, hole_lengths, chunk_data, *attributes], axis=1)

path_data = (
    await download_table(
        downloaded_dict["planned"]["path"],
        fb=FeedbackWidget(f"Downloading collar path data as '{downloaded_dict['planned']['path']['data']}'"),
    )
).to_pandas()

processed_path_data = pd.concat(
    [
        path_data.iloc[start : start + length].reset_index(drop=True)
        for start, length in zip(chunk_data["offset"], chunk_data["count"])
    ],
    axis=0,
).reset_index(drop=True)
hole_name = []

for hole_id, count in zip(chunk_data["hole_index"], chunk_data["count"]):
    hole_name.extend([index_to_name_map[index_to_name_map["key"] == hole_id]["value"].values[0]] * count)
processed_path_data["hole_name"] = hole_name
processed_path_data = processed_path_data[
    ["hole_name"] + [col for col in processed_path_data.columns if col != "hole_name"]
]

attribute_tables = {}
# Use the data client to download the attribute data and merge it with the coordinates data.
if "collections" in downloaded_dict["planned"]:
    for attribute_table in downloaded_dict["planned"]["collections"]:
        collection_type = attribute_table["collection_type"]
        collection_name = f"planned_{collection_type}_({attribute_table['name']})"

        if collection_type == "interval":
            distance_container = attribute_table["from_to"]["intervals"]["start_and_end"]
            attribute_container = attribute_table["from_to"]["attributes"]
            distance_data = (
                await data_client.download_table(
                    object_id=metadata.id,
                    version_id=None,
                    table_info=distance_container,
                    fb=FeedbackWidget(f"Downloading distance data as '{distance_container['data']}'"),
                )
            ).to_pandas()
        elif collection_type == "distance":
            distance_container = attribute_table["intervals"]["start_and_end"]
            attribute_container = attribute_table["distance"]["attributes"]
            distance_data = (
                await data_client.download_table(
                    object_id=metadata.id,
                    version_id=None,
                    table_info=distance_container,
                    fb=FeedbackWidget(f"Downloading distance data as '{distance_container['data']}'"),
                )
            ).to_pandas()
        else:
            continue

        attribute_chunk_data = (
            await data_client.download_table(
                object_id=metadata.id,
                version_id=None,
                table_info=attribute_table["holes"],
                fb=FeedbackWidget(f"Downloading attribute chunk data as '{attribute_table['holes']['data']}'"),
            )
        ).to_pandas()

        columns = [distance_data]
        for column in attribute_container:
            attribute_name = column["name"]
            attribute_type = column["attribute_type"]

            column_data = (
                await data_client.download_table(
                    object_id=metadata.id,
                    version_id=None,
                    table_info=column["values"],
                    fb=FeedbackWidget(
                        f"Downloading attribute '{attribute_name}' column data as '{column['values']['data']}'"
                    ),
                )
            ).to_pandas()
            column_data.columns = ["data"]

            # If the attribute is a category, download the 'table' data as well.
            if attribute_type == "category":
                lookup_table = (
                    await data_client.download_table(
                        object_id=metadata.id,
                        version_id=None,
                        table_info=column["table"],
                        fb=FeedbackWidget(
                            f"Downloading attribute '{attribute_name}' lookup table data as '{column['table']['data']}'"
                        ),
                    )
                ).to_pandas()

                # Merge the values data with the table data.
                merged_data = pd.merge(column_data, lookup_table, left_on="data", right_on="key", how="left")
                # Drop the 'data' and 'key' columns from the merged data.
                merged_data.drop(columns=["data", "key"], inplace=True)
                # Rename the 'value' column to the attribute name.
                merged_data.rename(columns={"value": attribute_name}, inplace=True)
                # Concatenate the merged data with the coordinates data.
                columns.append(merged_data)

            elif attribute_type == "scalar":
                # Rename the 'data' column to the attribute name.
                column_data.rename(columns={"data": attribute_name}, inplace=True)
                # Concatenate the data with the coordinates data.
                columns.append(column_data)

            else:
                raise ValueError(f"Unknown attribute type: {attribute_type}")

        attribute_tables[collection_name] = pd.concat(columns, axis=1)
        attribute_tables[f"{collection_name} (Cnk)"] = attribute_chunk_data

# Save the dataframe as a CSV file.
with pd.ExcelWriter(output_filename) as writer:
    df.to_excel(writer, sheet_name="Collars", index=False)
    path_data.to_excel(writer, sheet_name="Raw Paths", index=False)
    processed_path_data.to_excel(writer, sheet_name="Paths", index=False)
    for name, table in attribute_tables.items():
        sheet_name = name[:31] if len(name) > 31 else name
        table.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"✓ Drilling campaign data successfully saved to '{output_filename}'")